Now, let's finally look at that divorced dataset.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# from dateutil.relativedelta import relativedelta

In [2]:
df = pd.read_csv('../data/divorces_2000-2015_translated.csv')
print(df.shape)
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '')
df.head()

(4923, 41)


,divorce_date,type_of_divorce,nationality_partner_man,dob_partner_man,place_of_birth_partner_man,birth_municipality_of_partner_man,birth_federal_partner_man,birth_country_partner_man,age_partner_man,residence_municipality_partner_man,...,marriage_certificate_municipality,marriage_certificate_federal,level_of_education_partner_man,employment_status_partner_man,level_of_education_partner_woman,employment_status_partner_woman,marriage_duration,marriage_duration_months,num_children,custody
0,9/6/06,Necesario,MEXICANA,18/12/75,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,30.0,XALAPA,...,XALAPA,VERACRUZ,SECUNDARIA,OBRERO,SECUNDARIA,EMPLEADO,5.0,NaN,1.0,NaN
1,1/2/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,47.0,NaN,...,XALAPA,VERACRUZ,PREPARATORIA,ESTABLECIMIENTO,PREPARATORIA,EMPLEADO,NaN,NaN,NaN,NaN
2,1/2/05,Necesario,MEXICANA,22/2/55,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,49.0,NaN,...,XALAPA,VERACRUZ,PREPARATORIA,OBRERO,NaN,TRABAJADOR POR CUENTA PROPIA EN VIA PUBLICA,NaN,NaN,NaN,NaN
3,1/2/06,Necesario,MEXICANA,20/1/64,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,42.0,XALAPA,...,XALAPA,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,EMPLEADO,18.0,NaN,2.0,MADRE
4,1/2/06,Necesario,MEXICANA,30/10/75,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,30.0,COATEPEC,...,XALAPA,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,NO TRABAJA,7.0,NaN,2.0,MADRE


Yeah, once again we need to do a lot of sprucing. We'll definitely err on the side of making things as simple as possible.

In [3]:
df.isna().sum().sort_values(ascending=False)[:15]

marriage_duration_months               3368
custody                                2851
dob_registration_date_partner_woman    2679
monthly_income_partner_woman_peso      2119
num_children                           1912
monthly_income_partner_man_peso        1419
occupation_partner_woman                578
occupation_partner_man                  529
dob_partner_woman                       452
employment_status_partner_woman         417
dob_partner_man                         381
level_of_education_partner_woman        380
employment_status_partner_man           356
residence_municipality_partner_man      324
residence_country_partner_man           324
dtype: int64

Oh my, that's quite a lot of nulls. But, let us deal with each feature as we get around to it. Especially when our target is either the duration or the type...

In [4]:
 i = 0

In [5]:
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts() #Yeah, we don't care about this... way too detailed for our purposes. Do note though
#the range of them

Now looking at feature 0.


divorce_date
9/10/09     9
5/9/05      8
25/2/09     8
21/11/03    8
3/11/04     8
           ..
8/10/13     1
8/10/14     1
8/11/01     1
8/11/06     1
31/12/13    1
Name: count, Length: 2596, dtype: int64

In [6]:
#Now that EDA has been done (not reflective of this workbook's presentation) we'll already calculate the length of marriage.

# df['start_date'] = pd.to_datetime(df['divorce_date'])
# df['end_date'] = pd.to_datetime(df['m'])

# df['marriage_duration'] = (df['end_date'] - df['start_date']).dt.days
# df['marriage_duration'] = df['marriage_duration'] / 365.25

#Oh wait - gf me... there is no formal dom date. Hence, I suppose we'll need to do the following. Similarily, since we never formally
#Received their dom we'll infer their ages a that time by subtracting the marriage duration from their age at date of divorce.

#I feel like this is a bit clunky, but eh - this is a "passion project" of sorts...

In [7]:
df['divorce_date'].isnull().sum(), df['marriage_duration'].isnull().sum(), df['marriage_duration_months'].isnull().sum()

(0, 235, 3368)

In [8]:
df['marriage_duration'].fillna(0, inplace=True)
df['marriage_duration_months'].fillna(0, inplace=True)

In [9]:
df['marrige_length'] = round(df['marriage_duration'] + df['marriage_duration_months']/12, 3)

In [10]:
df['marrige_length']

0        5.000
1        0.000
2        0.000
3       18.000
4        7.000
         ...  
4918     3.917
4919     1.000
4920    35.000
4921    33.000
4922     4.000
Name: marrige_length, Length: 4923, dtype: float64

In [11]:
df['marrige_length'].describe()

count    4923.000000
mean       11.329597
std         9.387950
min         0.000000
25%         4.000000
50%         9.000000
75%        16.125000
max        61.000000
Name: marrige_length, dtype: float64

A bit informal right now, but let's do it anyways:

Nice bit of skew to the right. Quite tempted to see the effect children have on this, but for now we continue.

In [12]:
df[df['marrige_length']==0] #Oh my... eh, we'll deal with these later.

,divorce_date,type_of_divorce,nationality_partner_man,dob_partner_man,place_of_birth_partner_man,birth_municipality_of_partner_man,birth_federal_partner_man,birth_country_partner_man,age_partner_man,residence_municipality_partner_man,...,marriage_certificate_federal,level_of_education_partner_man,employment_status_partner_man,level_of_education_partner_woman,employment_status_partner_woman,marriage_duration,marriage_duration_months,num_children,custody,marrige_length
1,1/2/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,47.0,NaN,...,VERACRUZ,PREPARATORIA,ESTABLECIMIENTO,PREPARATORIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
2,1/2/05,Necesario,MEXICANA,22/2/55,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,49.0,NaN,...,VERACRUZ,PREPARATORIA,OBRERO,NaN,TRABAJADOR POR CUENTA PROPIA EN VIA PUBLICA,0.0,0.0,NaN,NaN,0.0
16,1/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,24.0,NaN,...,VERACRUZ,SECUNDARIA,EMPLEADO,SECUNDARIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
21,1/3/05,Necesario,MEXICANA,17/8/48,EL ESPINAL,ESPINAL,VERACRUZ,MEXICO,56.0,XALAPA,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
45,1/6/04,Necesario,MEXICANA,1/5/48,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,56.0,NaN,...,VERACRUZ,PRIMARIA,EMPLEADO,PRIMARIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,31/1/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,56.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
4849,31/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,31.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
4850,31/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,58.0,NaN,...,VERACRUZ,PROFESIONAL,PATRON O EMPRESARIO,SECUNDARIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
4851,31/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,47.0,NaN,...,VERACRUZ,PREPARATORIA,EMPLEADO,PRIMARIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0


In [13]:
df[(df['marrige_length']==0) & (df['dob_partner_man'].isnull())] #Hmm, perhaps a trend. But for now...

,divorce_date,type_of_divorce,nationality_partner_man,dob_partner_man,place_of_birth_partner_man,birth_municipality_of_partner_man,birth_federal_partner_man,birth_country_partner_man,age_partner_man,residence_municipality_partner_man,...,marriage_certificate_federal,level_of_education_partner_man,employment_status_partner_man,level_of_education_partner_woman,employment_status_partner_woman,marriage_duration,marriage_duration_months,num_children,custody,marrige_length
1,1/2/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,47.0,NaN,...,VERACRUZ,PREPARATORIA,ESTABLECIMIENTO,PREPARATORIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
16,1/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,24.0,NaN,...,VERACRUZ,SECUNDARIA,EMPLEADO,SECUNDARIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
49,1/6/05,Necesario,MEXICANA,NaN,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,35.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
70,1/7/05,Necesario,MEXICANA,NaN,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,28.0,XALAPA,...,VERACRUZ,PREPARATORIA,EMPLEADO,PREPARATORIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
152,2/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,64.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4832,31/1/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,60.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
4833,31/1/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,56.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
4849,31/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,31.0,NaN,...,VERACRUZ,PROFESIONAL,EMPLEADO,PROFESIONAL,EMPLEADO,0.0,0.0,NaN,NaN,0.0
4850,31/3/00,Voluntario,MEXICANA,NaN,NaN,NaN,NaN,NaN,58.0,NaN,...,VERACRUZ,PROFESIONAL,PATRON O EMPRESARIO,SECUNDARIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0


In [14]:
print(df['dob_partner_man'].isnull().sum(), df['dob_partner_woman'].isnull().sum())
print(df['age_partner_man'].isnull().sum(), df['age_partner_woman'].isnull().sum())

381 452
107 151


In [15]:
#Given that ages have less nulls we'll go with them, albeit potentially losing accuracy via not calculatng the ages at marriage
#via their dobs.

#Note that now that I'm subtracting the marriage length already my comments later on might be a bit off as they are
#be reflective of the original data where age reflects dod.

#Note that we'll still have to deal with nulls later. But for now:

df['age_partner_man'] = df['age_partner_man'] - df['marrige_length']
df['age_partner_woman'] = df['age_partner_woman'] - df['marrige_length']

In [16]:
to_be_deleted = []

to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is divorce_date and now we're poised to delete a total of 1 from our dataframe.


In [17]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 1.


type_of_divorce
Necesario     2528
Voluntario    2395
Name: count, dtype: int64

Guess we'll worry about one-hot-encodng later.

Pragmatically, I'd assume Necesario should be 0. Ie the type of scenario we'd ideally like to avoid, but relatively 'better' than the alternative (in short...).

Like we've written about (somewhere), this is potentially bleeding with the target. Then again, if we work with the assumption/qualify the results of this study such that, "Given a couple's marriage fails, how long will it last?" So, that issue wouldn't apply. Similarily, perhaps neccesary/voluntary can approximate (albeit incredibly losely) the state of the marital harmony/how wretched was it - whose data we don't have.

In [18]:
df[df.columns[i]] = df[df.columns[i]].apply(lambda x: 0 if x == 'Necesario' else 1)

In [19]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 2.


nationality_partner_man
MEXICANA          4879
CUBANA               8
ESTADOUNIDENSE       7
ESPA√ëOLA            6
ARGENTINA            4
FRANCESA             2
CANADIENSE           2
ALEMANA              2
VENEZOLANA           2
JAPONESA             1
AUSTRALIANA          1
POLACA               1
CHINA                1
ITALIANA             1
CHILENA              1
COSTARRICENSE        1
AUSTRIACA            1
COLOMBIANA           1
NICARAGUENSE         1
Name: count, dtype: int64

Yeah, we'll simplify this to mexican/not. Although we'd be interested to see a mexican of any type with a non, we're also interested in a case where neither are mexican yet are still living there (if it occurs in this dataset).

In [20]:
df[df.columns[i]] = df[df.columns[i]].apply(lambda x: 0 if x == 'MEXICANA' else 1)

In [21]:
df.head()

,divorce_date,type_of_divorce,nationality_partner_man,dob_partner_man,place_of_birth_partner_man,birth_municipality_of_partner_man,birth_federal_partner_man,birth_country_partner_man,age_partner_man,residence_municipality_partner_man,...,marriage_certificate_federal,level_of_education_partner_man,employment_status_partner_man,level_of_education_partner_woman,employment_status_partner_woman,marriage_duration,marriage_duration_months,num_children,custody,marrige_length
0,9/6/06,0,0,18/12/75,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,25.0,XALAPA,...,VERACRUZ,SECUNDARIA,OBRERO,SECUNDARIA,EMPLEADO,5.0,0.0,1.0,NaN,5.0
1,1/2/00,1,0,NaN,NaN,NaN,NaN,NaN,47.0,NaN,...,VERACRUZ,PREPARATORIA,ESTABLECIMIENTO,PREPARATORIA,EMPLEADO,0.0,0.0,NaN,NaN,0.0
2,1/2/05,0,0,22/2/55,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,49.0,NaN,...,VERACRUZ,PREPARATORIA,OBRERO,NaN,TRABAJADOR POR CUENTA PROPIA EN VIA PUBLICA,0.0,0.0,NaN,NaN,0.0
3,1/2/06,0,0,20/1/64,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,24.0,XALAPA,...,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,EMPLEADO,18.0,0.0,2.0,MADRE,18.0
4,1/2/06,0,0,30/10/75,XALAPA - ENRIQUEZ,XALAPA,VERACRUZ,MEXICO,23.0,COATEPEC,...,VERACRUZ,PROFESIONAL,EMPLEADO,PREPARATORIA,NO TRABAJA,7.0,0.0,2.0,MADRE,7.0


In [22]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 3.


dob_partner_man
25/5/73     5
5/4/74      5
17/3/75     4
23/4/74     4
2/8/61      4
           ..
1/10/72     1
19/6/68     1
5/3/70      1
20/12/77    1
7/2/58      1
Name: count, Length: 3848, dtype: int64

Hmm, potentially let us simplify.... Oh wait, I just ealized - I want to example their relative ages.Hence, I'd need tha divocre data ultimately.... we'll keep on going for now though.

In [23]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is dob_partner_man and now we're poised to delete a total of 2 from our dataframe.


In [24]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 4.


place_of_birth_partner_man
XALAPA - ENRIQUEZ    2472
DISTRITO FEDERAL      347
VERACRUZ              129
COATEPEC               68
POZA RICA              49
                     ... 
ANTONIO HIDALGO         1
ESPARTACO               1
AHUACATAN               1
SAN PABLO COAPAN        1
EL LIMON                1
Name: count, Length: 669, dtype: int64

Yeah, we're simplifyng this. Let the cut off be 10 (beyond the scope to comment here why I'm chosing that number).

In [25]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is place_of_birth_partner_man and now we're poised to delete a total of 3 from our dataframe.


In [26]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 5.


birth_municipality_of_partner_man
XALAPA                   2479
DISTRITO FEDERAL          348
VERACRUZ                  131
COATEPEC                   78
MARTINEZ DE LA TORRE       60
                         ... 
PATZCUARO                   1
TOPILITOS DE ZARAGOZA       1
TULA DE ALLENDE             1
PARRAL                      1
ZAUTLA                      1
Name: count, Length: 424, dtype: int64

Somehow different??? A bit tempted to delete as a quick scan shows barely any differencs. However, this seems more inclusive so I'll go with this (424 vs. 669).

In [27]:
list(df[df.columns[i]].value_counts(normalize=True)[:9].index)

['XALAPA',
 'DISTRITO FEDERAL',
 'VERACRUZ',
 'COATEPEC',
 'MARTINEZ DE LA TORRE',
 'MISANTLA',
 'POZA RICA',
 'ORIZABA',
 'CORDOBA']

In [28]:
print(sum(df[df.columns[i]].value_counts(normalize=True)[:9]))

0.6846057571964955


In [29]:
#I feel like I had a neat code to do this. Oh Well.

feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:9].index)

#Somewhat arbitrarily chosen, but eh. After like 7 they have such a small amount... Will simplify.

df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [30]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 6.


birth_federal_partner_man
VERACRUZ              3920
DISTRITO FEDERAL       403
PUEBLA                 111
TAMAULIPAS              33
OAXACA                  30
                      ... 
EL SALVADOR              1
GALICIA                  1
MISSOURI                 1
CAMAG√úEY                1
NORTE DE SANTANDER       1
Name: count, Length: 76, dtype: int64

Similar to the other.... We might as well keep it Especially since we're condensing. This time we'll simplify it to Puebla than 'Other' as they don' even have 100...

In [31]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:3].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [32]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 7.


birth_country_partner_man
MEXICO                       4748
CUBA                            8
ESPA√ëA                         8
ESTADOS UNIDOS DE AMERICA       6
ARGENTINA                       3
COLOMBIA                        3
FRANCIA                         3
VENEZUELA                       2
POLONIA                         2
ALEMANIA                        2
COSTA RICA                      1
CANADA                          1
CENTRO AMERICA                  1
CHILE                           1
IRAN                            1
ITALIA                          1
JAPON                           1
MARYLAND                        1
INGLATERRA                      1
AUSTRALIA                       1
NICARAGUA                       1
Name: count, dtype: int64

More spread than then similar feature earlier of nationality. So, there's approximately 100ish 'Mexicans' born outside of Mexico that later returned. Potentially relevaant, but eh, I'd rather keep things simpler.

In [33]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is birth_country_partner_man and now we're poised to delete a total of 4 from our dataframe.


In [34]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 8.


age_partner_man
25.000    241
28.000    232
24.000    226
26.000    217
27.000    217
         ... 
39.833      1
32.750      1
51.333      1
55.667      1
44.750      1
Name: count, Length: 384, dtype: int64

I just reconfirmed and yes this is referring to dod. Hence, we're good to go to simplify by removing (as we originally assumed) the dob. However, I'd still be interested in seeing about how old they were when they got married. So, we'll see.

Recall that in the current version of things we have subtracted the duratio of their marriage. So, this feature approximates (as ultimately I have some reservations about the data, rounding, etc.) their dom.

In [35]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 9.


residence_municipality_partner_man
XALAPA                 3927
VERACRUZ                 67
COATEPEC                 63
EMILIANO ZAPATA          62
DISTRITO FEDERAL         40
                       ... 
SEBASTIAN TUXTLA          1
GUSTAVO A. MADERO         1
PETLALCINGO               1
TECOLUTILLA               1
SANTA MARIA EL TULE       1
Name: count, Length: 175, dtype: int64

In [36]:
df[df.columns[i]].value_counts(normalize=True)[:4]

residence_municipality_partner_man
XALAPA             0.853881
VERACRUZ           0.014568
COATEPEC           0.013699
EMILIANO ZAPATA    0.013481
Name: proportion, dtype: float64

Eh... who cares. We'll keep it. Simplify it to anything past Emiliana Zapata ak 1 percent is other.

In [37]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:4].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [38]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()[:5]

Now looking at feature 10.


residence_federal_partner_man
VERACRUZ            4402
DISTRITO FEDERAL      53
PUEBLA                27
TABASCO               11
CHIAPAS                8
Name: count, dtype: int64

Yeah, this is becoming binary. More spread out than the 'birth_country_partner_man' so worth having both.

In [39]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:1].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [40]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 11.


residence_country_partner_man
MEXICO                       4589
ESTADOS UNIDOS DE AMERICA       7
ESPA√ëA                         1
AUSTRALIA                       1
PUERTO RICO                     1
Name: count, dtype: int64

Eh, a lot of these are similar... too bad. We'll keep them anyways. At least for now.

In [41]:
#Might as well use the same code, despite it's prime use for multiple guys:

feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:1].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [42]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 12.


monthly_income_partner_man_peso
5000.0     326
3000.0     301
4000.0     279
10000.0    276
6000.0     275
          ... 
95000.0      1
6300.0       1
4100.0       1
550.0        1
12500.0      1
Name: count, Length: 214, dtype: int64


Hmm,pretty small after the first one. Potentially a move to just make this binary. At least for now we'll conntinue.

Heads up already re. potential inflation issues. However, the data isn't too far apart so we'll likely be lax and not bother for the sake of this study.

Perhaps if it turns out that this feature is super significant we'll reconsider.

Note that I'm assuming this is referring to the income at the dod. Hence, a bit of a weird interaction will be at place when their ages reflect the beginning of their marriage; this at the end.

In [43]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 13.


occupation_partner_man
EMPLEADO                0.535275
COMERCIANTE             0.068047
PROFESOR                0.030041
MAESTRO                 0.020255
MEDICO                  0.019800
                          ...   
AGENTE DE SEGUROS       0.000228
PRODUCTOR               0.000228
ADMINISTRATIVO          0.000228
MEDICO ANESTECIOLOGO    0.000228
CAPITAN DE CORBETA      0.000228
Name: proportion, Length: 221, dtype: float64

First 4 are more than 2%; 13 are 1% so at most do 13 than an other category. They sum up to 65 and 80%.

Similar to things we've seen before, but eh...

In [45]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:3].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers else i for i in df[df.columns[i]]]

#Reminder that we're also filled in nulls with this method. Hence, percentages of the preserved groups will decrease.

In [49]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=False)

Now looking at feature 14.


place_of_residence_partner_man
XALAPA-ENRIQUEZ     3918
VERACRUZ              66
COATEPEC              57
EMILIANO ZAPATA       37
DISTRITO FEDERAL      34
                    ... 
ZONGOLICA              1
CERRO COLORADO         1
MIGUEL HIDALGO         1
ALVARO OBREGON         1
LAS PUENTES            1
Name: count, Length: 229, dtype: int64

In [50]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:1].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

## Re. the women:

By default won't bother to comment unless I notice trends that I find significant between the two. Ie even if the nationality is much more likely to be Mexican if a woman I won't bother commenting given that they're similar.

In [51]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 15.


nationality_partner_woman
MEXICANA          4890
CUBANA               6
ESTADOUNIDENSE       5
FRANCESA             3
ESPA√ëOLA            3
ARGENTINA            2
POLACA               2
BRASILE√ëA           1
PERUANA              1
DOMINICANA           1
VENEZOLANA           1
COLOMBIANA           1
BOLIVIANA            1
JAPONESA             1
BRITANICA            1
SUIZA                1
Name: count, dtype: int64

In [52]:
df[df.columns[i]] = df[df.columns[i]].apply(lambda x: 0 if x == 'MEXICANA' else 1)

In [53]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 16.


dob_partner_woman
11/12/72    5
19/2/71     5
6/6/68      4
6/7/77      4
3/6/72      4
           ..
28/2/65     1
11/12/71    1
1/10/52     1
25/3/76     1
22/1/72     1
Name: count, Length: 3765, dtype: int64

Like earlier with the man, it's want to be deleted. But, let's continue for now.

In [54]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is dob_partner_woman and now we're poised to delete a total of 5 from our dataframe.


In [55]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 17.


dob_registration_date_partner_woman
21/2/73     4
1/8/77      3
7/7/69      3
3/5/73      3
8/6/70      3
           ..
16/2/52     1
31/8/74     1
29/5/87     1
27/12/76    1
7/9/72      1
Name: count, Length: 2003, dtype: int64

Woah, we did not have this earlier. More than half nulls and not even sure what it is. Perhaps a proxy of some kind for growing up in a messed up home if the dob somehow wasn't legally/formally recognized in a hospital. Really do not know. But yeah, we're deleting this one.

In [ ]:
# j = -1

In [ ]:
# j += 1
# print(j)
# df.iloc[j]

In [56]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is dob_registration_date_partner_woman and now we're poised to delete a total of 6 from our dataframe.


In [57]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 18.


place_of_birth_partner_woman
XALAPA-ENRIQUEZ         2595
DISTRITO FEDERAL         306
VERACRUZ                 107
POZA RICA                 54
MARTINEZ DE LA TORRE      51
                        ... 
EL TIBOR                   1
TLATLAUQUITEPEC            1
HUAYACOCOTLA               1
MIRADORES DEL MAR          1
LA LIMA                    1
Name: count, Length: 653, dtype: int64

Like above... Besides we're deletign something I'll actually comment though.

In [58]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is place_of_birth_partner_woman and now we're poised to delete a total of 7 from our dataframe.


In [59]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 19.


birth_municipality_of_partner_woman
XALAPA                  2612
DISTRITO FEDERAL         308
VERACRUZ                 110
MARTINEZ DE LA TORRE      66
POZA RICA                 55
                        ... 
IRAPUATO                   1
OTATITLAN                  1
IXTACZOQUITLAN             1
JESUS CARRANZA             1
PONTEVEDRA                 1
Name: count, Length: 404, dtype: int64

In [60]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:9].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [61]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 20.


birth_federal_partner_woman
VERACRUZ            4011
DISTRITO FEDERAL     362
PUEBLA                93
OAXACA                38
TAMAULIPAS            30
                    ... 
SANTO DOMINGO          1
NUEVA ESPARTA          1
SAN PAULO              1
ESPA√ëA                1
ZURICH                 1
Name: count, Length: 72, dtype: int64

In [62]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:3].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [63]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 21.


birth_country_partner_woman
MEXICO                       4742
ESTADOS UNIDOS DE AMERICA       7
CUBA                            7
ESPA√ëA                         4
FRANCIA                         3
REPUBLICA DOMINICANA            3
BRASIL                          2
COLOMBIA                        2
SUIZA                           2
POLONIA                         2
URUGUAY                         1
ARGELIA                         1
JAPON                           1
ARGENTINA                       1
VENEZUELA                       1
GUATEMALA                       1
BOLIVIA                         1
CALIFORNIA                      1
PERU                            1
INGLATERRA                      1
Name: count, dtype: int64

In [64]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Next up to be deleted is birth_country_partner_woman and now we're poised to delete a total of 8 from our dataframe.


In [65]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 22.


age_partner_woman
 24.000    260
 25.000    253
 23.000    234
 26.000    211
 21.000    207
          ... 
 14.083      1
 54.500      1
 14.167      1
 9.500       1
-0.667       1
Name: count, Length: 390, dtype: int64

Oh, we have a negative age... That's at best a mistake. So, guess we'll replace with median there. On the topic - I seem to be noticing some child brides... albeit very few so likely true so I won't worry about it.

In [66]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 23.


place_of_residence_partner_woman
XALAPA-ENRIQUEZ    4162
EMILIANO ZAPATA      43
COATEPEC             40
VERACRUZ             35
BANDERILLA           26
                   ... 
CHILTOYAC             1
EL ESPINAL            1
VILLA RICA            1
INDIANA               1
SANTIAGO TUXTLA       1
Name: count, Length: 174, dtype: int64

In [ ]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:1].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [67]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 24.


residence_municipality_partner_woman
XALAPA                4179
EMILIANO ZAPATA         61
COATEPEC                41
VERACRUZ                37
BANDERILLA              26
                      ... 
PACHUCA                  1
TRES VALLES              1
VALENCIA                 1
CUAUTITLAN IZCALLI       1
SANTIAGO TUXTLA          1
Name: count, Length: 131, dtype: int64

In [68]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:4].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [69]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 25.


residence_federal_partner_woman
VERACRUZ                4489
DISTRITO FEDERAL          25
PUEBLA                    13
QUINTANA ROO               9
QUERETARO                  7
TAMAULIPAS                 6
MEXICO                     6
ESTADO DE MEXICO           6
MORELOS                    6
CHIAPAS                    5
CAMPECHE                   5
BAJA CALIFORNIA            5
OAXACA                     4
TABASCO                    4
HIDALGO                    4
AGUASCALIENTES             3
GUANAJUATO                 2
SONORA                     2
GUERRERO                   2
YUCATAN                    2
NAYARIT                    1
CHAUTEMPAN                 1
PARIS                      1
TEXAS                      1
FLORIDA                    1
USA                        1
NEW YORK                   1
COMUNIDAD VALENCIANA       1
BAJA CALIFORNIA SUR        1
MICHOACAN                  1
COLIMA                     1
TEXCOCO                    1
COAHUILA                   1
Name: count

Perhaps due earlier and already tacitly assumed, but whatever proxy for these I'll use for both the men and women.

In [70]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:1].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [71]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Now looking at feature 26.


residence_country_partner_woman
MEXICO            4612
ESTADOS UNIDOS       4
ESPA√ëA              1
FRANCIA              1
Name: count, dtype: int64

In [ ]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:1].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [73]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 27.


occupation_partner_woman
EMPLEADA              0.434062
AMA DE CASA           0.126582
LABORES DOMESTICAS    0.074799
MAESTRA               0.049482
COMERCIANTE           0.040046
                        ...   
SENADORA              0.000230
BANQUERO              0.000230
SOCIOLOGA             0.000230
MODESTA               0.000230
PREPARATORIA          0.000230
Name: proportion, Length: 156, dtype: float64

Much more of a spread - unsurprisingly per 'domain knowledge.' Match ups wil be interesting.

In [ ]:
sum(df[df.columns[i]].value_counts(normalize=True)[:3]), sum(df[df.columns[i]].value_counts(normalize=True)[:10])

In [ ]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:3].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers else i for i in df[df.columns[i]]]

In [ ]:
stopstop

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

In [ ]:
feature_keepers = list(df[df.columns[i]].value_counts(normalize=True)[:3].index)
df[df.columns[i]] = ["OTHER" if i not in feature_keepers  else i for i in df[df.columns[i]]]

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Oh! What we've been waiting for. In of itself we don't care, but releative to: Man's age, woman's, date of divorce - it's very important!

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

A potentially new factor. Not necessarily the same place where the marriage ceremony, if any, took place, but likely a potential factor. Oh the stories with notaries, pre-nuptials, etc. ... But such are definitely beyond the scope of this project.

Somewhat tangentially: Did I delete some code? Had to copy more code the next day for this column... Oh well.

In [ ]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Like other things, very similar... So, we'll go with the more general feeature.

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

In [ ]:
len(df[df.columns[i]].value_counts())

In [ ]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Hmm, this time I'll drop this one. Although I'm planning to simplify the features, I'd rather have more a spread than not.

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Nice to see a 'contained' feature.

I might be inclined to make this hierarchical - puting 'sin' as 0 then I suppose 'otro' as 1 or .5 then the rest correspondingly. We'll see.

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Likely we've made this type of comment before, but reminder to make a feature representing employment interaction between spouses.

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()[:12]

I'm only planning on having one duration column. So, we'll do math stuff in a bit to see if we'll even bother including years. Pragmatically, might as well make it simple and do years; eh, if we decide to do regression/make our target marrige duration then estimations would likely be, def., regression-based hence getting into fractions/months so we might as well make it more accurate.

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Oh wait, the previous one was years?!?! Regardless, same idea applies.

Likewise, I'll assume a null is 0 and base the length of their marriage just on years.

In [ ]:
sum(df[df.columns[i]].value_counts())

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

Oooh, quite looking forward to seeing how this will affect the data. A bit tempted to perhaps group 5+... We'll see. For now we'll leave as is.

Note that this category had many nulls. I'll assume in such cases that the couple had no children.

In [ ]:
i += 1
print(f"Now looking at feature {i}.")
df[df.columns[i]].value_counts()

And finally, the ultimate feature:

If we'd go with, which I'm thinking more and more not, to do wih type of divorce as the target then likely this may be quite important. However, I don't think I am. Hence, I don't care about custody as that's an after-the-fact issue ultimately. Plus, with so many nulls; perhaps it means joint; eh, still null...

In [ ]:
to_be_deleted.append(df.columns[i])
print(f"Next up to be deleted is {df.columns[i]} and now we're poised to delete a total of {len(to_be_deleted)} from our dataframe.")

Re. check with above whatever feeatures we condensed with men to do the same with women (per 9/3/24 just doing eda essentially).

OHE:

type_of_divorce

?bucket monthly incomes?

occupation


Fix: Men's employment...

??? Re. education to make it hierarchical as opposed to OHE

Add to the deletions: months, years, etc. --> after we calculate for ourselves the marriage duration (assuming we go with marriage duration as our target).

Re. analyze he various data that has a marriag duration of 0...

Confirm accuracy of age data. Ie no negative ages...